## Reinforcement Learning - Aoudia and Hoydis
##### Imports

In [10]:
import tensorflow as tf
import keras 
import numpy as np
from keras.layers import Input, Dense, Lambda, ELU
from keras.models import Model
from keras.layers.normalization import BatchNormalization 
from keras import backend as K
from keras.layers import GaussianNoise, advanced_activations
from keras.engine.topology import Layer
from keras.legacy import interfaces
from keras.initializers import Zeros as kZeros
from keras.utils import multi_gpu_model
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import load_model
from keras.callbacks import EarlyStopping
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
# from tqdm import tqdm_notebook, tnrange
from time import time
import pickle

In [2]:
# # confirm TensorFlow sees the GPU
# from tensorflow.python.client import device_lib
# assert 'GPU' in str(device_lib.list_local_devices())

# # confirm Keras sees the GPU
# from keras import backend
# assert len(backend.tensorflow_backend._get_available_gpus()) > 0

In [3]:
# This is in a seperate box because it isn't running on the 
# AWS server. 
from tqdm import tqdm_notebook, tnrange

##### Useful guides
https://hub.packtpub.com/build-reinforcement-learning-agent-in-keras-tutorial/
<br>
https://medium.com/ml-everything/policy-based-reinforcement-learning-with-keras-4996015a0b1
<br>

##### Notes from paper
- Loss function = Cross Entropy
- Normalisation = Average L2 power constraint = 1. $\mathbb{E}\left[\frac{1}{N}\left\Vert x\right\Vert_2^2\right]$ = 1.
- Trained at an SNR of 10dB for AWGN and an SNR of 20dB for RBF channel. $\sigma_{\pi}^2$ = 0.02 at training time.
    - During RL exploration $\mathbf{x_p} = \mathbf{x} + \mathbf{w}$, where each element of $\mathbf{w}$ is i.i.d ~ $\mathcal{N}(0,\sigma_{\pi}^2)$
- M= 256, N=4. N= the number of complex channel uses, so n = 8. Therefore n,k = (8,8)
- AWGN -> 1 hidden layer, size M, ReLu activation function
- Rayleigh -> see diagram.
    - Two layers (Dense(20,tanh)->Dense(2,linear)) calculate an estimate for $\hat{h}$, then we divide the received signal by $\hat{h}$ then have two layers for finding the received signal. Dense(M,ReLu) then Dense(M,Softmax). Then have the select maximum likelihood symbol layer.
- SNR $ = \frac{\mathbb{E}\left[\frac{1}{N}\left\Vert x\right\Vert_2^2\right]}{\sigma^2}$, but because of the normalisation this is $\frac{1}{\sigma^2}$.
- The RBF seems to be slow fading, but I could check for fast fading as well. Rayleigh fading is of the form $\mathbf{r} = \mathbf{s}*\mathbf{h} + \mathbf{n}$ where $\mathbf{h}$ ~ $\mathcal{N}(0,\sigma_m^2)$, $\mathbf{n}$ ~ $\mathcal{N}(0,\sigma_a^2)$. 
    - Slow fading: h stays the same across the whole minibatch
    - Fast fading: h changes every sample
- I'm going to guess that $\sigma_m ~ \frac{1}{3}$. Because this means that 98% of the time it's less than 1. Which sounds alright.

##### Work Log

27/05/2019
- Started researching RBF, think I can implement it in a custom layer.
- Made lots of notes
- Copied over functions from other notebook.

### Function Definitions

In [4]:
def most_likely(posterior_probs):
    max_vals = K.max(posterior_probs, axis=1, keepdims=True) 
    max_vals = K.cast(max_vals, 'float32')
    geT = K.greater_equal(posterior_probs, max_vals)
    return K.cast(geT, 'float32')

In [5]:
class MostLikelySymbol(Layer):
    """Return the most likely symbol from a softmax input in the
    one hot encoded form.

    This layer is only active at test time as otherwise it would
    stop gradient propogation during training. Also it is useful
    to train with a softmax output to encourage a decisive 
    decision and because it means you can assess confidence.

    # Arguments
        None

    # Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.

    # Output shape
        Same shape as input.
    """

    @interfaces.legacy_gaussiannoise_support
    def __init__(self, **kwargs):
        super(MostLikelySymbol, self).__init__(**kwargs)
        self.supports_masking = True

    def call(self, inputs, training=None):
        def most_likely():
            max_vals = K.max(inputs, axis=1, keepdims=True) 
            max_vals = K.cast(max_vals, 'float32')
            geT = K.greater_equal(inputs, max_vals)
            return K.cast(geT, 'float32')            
        return K.in_train_phase(inputs, most_likely, training=training)

    def get_config(self):
        config = {}
        base_config = super(MostLikelySymbol, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape

In [6]:
class GaussianNoise2(Layer):
    """Apply additive zero-centered Gaussian noise at both traning
    and test time.

    This is useful to mitigate overfitting
    (you could see it as a form of random data augmentation).
    Gaussian Noise (GS) is a natural choice as corruption process
    for real valued inputs.

    Unlike the built in GaussianNoise regularisation layer it is 
    active at both training and test time. 

    # Arguments
        stddev: float, standard deviation of the noise distribution.

    # Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.

    # Output shape
        Same shape as input.
    """

    @interfaces.legacy_gaussiannoise_support
    def __init__(self, stddev, **kwargs):
        super(GaussianNoise2, self).__init__(**kwargs)
        self.supports_masking = True
        self.stddev = stddev

    def call(self, inputs, training=None):
        def noised():
            return inputs + K.random_normal(shape=K.shape(inputs),
                                            mean=0.,
                                            stddev=self.stddev)
        return K.in_train_phase(noised, noised, training=training)

    def get_config(self):
        config = {'stddev': self.stddev}
        base_config = super(GaussianNoise2, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape

In [169]:
def custom_mult(in_,col_sel,h):
    tmp = K.tf.multiply(col_sel,h)
    tmp_expand = K.expand_dims(tmp,axis=1)
    return K.tf.multiply(in_,tmp_expand)

def complex_mult(in_,h):
    o1 = K.constant(np.array([1,0]))
    o2 = K.constant(np.array([0,1]))
    h_swap = K.tf.reverse(h,[1])
    in_swap = K.tf.reverse(in_,[2])

    t1 = custom_mult(in_,o1,h)          # real*real
    t2 = custom_mult(in_swap,o1,h_swap) # imaginary*imaginary
    t3 = custom_mult(in_swap,o2,h)      # real*imaginary
    t4 = custom_mult(in_,o2,h_swap)     # imaginary*real

    total1 = K.tf.math.subtract(t1,t2)  # r1*r2-i1*i2
    total2 = K.tf.math.add(t3,t4)       # r1*i2 + i1*r2
    return K.tf.math.add(total1,total2) # r1*r2-i1*i2 + r1*i2 + i1*r2

sigma_m = 0.33
sigma_a = 1

def RBF_fade(in_, sigma_m, sigma_a):
    hT = K.random_normal(shape=(K.shape(in_)[0],K.shape(in_)[2]),
                         mean=0.,
                         stddev=sigma_m)
    nT = K.random_normal(shape=K.shape(in_),
                         mean=0.,
                         stddev=sigma_a)
    return K.tf.add(complex_mult(in_,hT),nT)

In [91]:
in_ = np.random.rand(5,1,2)
in_T = K.constant(in_)
sess = tf.Session()
with sess.as_default():
    in_T.eval(session=sess)
in_T.eval(session=sess)

array([[[0.2953959 , 0.9147259 ]],

       [[0.80803007, 0.83845335]],

       [[0.3660615 , 0.46681798]],

       [[0.7871175 , 0.38486257]],

       [[0.73017544, 0.7195798 ]]], dtype=float32)

In [198]:
class RayleighBlockFading_fast(Layer):
    """
    Applies Rayleigh Block (fast) Fading to the input data at both 
    training and test time.

    # Arguments
        sigma_m: float, standard deviation of the multiplicative 
        constant noise distribution.
        sigma_a: float, standard deviation of the additive 
        constant noise distribution.

    # Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.

    # Output shape
        Same shape as input.
    """

    def __init__(self, sigma_m, sigma_a, **kwargs):
        super(RayleighBlockFading_fast, self).__init__(**kwargs)
        self.supports_masking = True
        self.sigma_m = sigma_m
        self.sigma_a = sigma_a

    def call(self, inputs, training=None):   
        def custom_mult(in_,col_sel,h):
            tmp = K.tf.multiply(col_sel,h)
            tmp_expand = K.expand_dims(tmp,axis=1)
            return K.tf.multiply(in_,tmp_expand)
        def complex_mult(in_,h):
            o1 = K.constant(np.array([1,0]))
            o2 = K.constant(np.array([0,1]))
            h_swap = K.tf.reverse(h,[1])
            in_swap = K.tf.reverse(in_,[2])

            t1 = custom_mult(in_,o1,h)          # real*real
            t2 = custom_mult(in_swap,o1,h_swap) # imaginary*imaginary
            t3 = custom_mult(in_swap,o2,h)      # real*imaginary
            t4 = custom_mult(in_,o2,h_swap)     # imaginary*real

            total1 = K.tf.math.subtract(t1,t2)  # r1*r2-i1*i2
            total2 = K.tf.math.add(t3,t4)       # r1*i2 + i1*r2
            return K.tf.math.add(total1,total2) # r1*r2-i1*i2 + r1*i2 + i1*r2
        def RBF_fade():
            hT = K.random_normal(shape=(K.shape(inputs)[0],K.shape(inputs)[2]),
                                 mean=0.,
                                 stddev=self.sigma_m)
            nT = K.random_normal(shape=K.shape(inputs),
                                 mean=0.,
                                 stddev=self.sigma_a)
            return K.tf.add(complex_mult(inputs,hT),nT)
        return K.in_train_phase(RBF_fade, RBF_fade, training=training)

    def get_config(self):
        config = {'sigma_m': self.sigma_m, 'sigma_a': self.sigma_a}
        base_config = super(RayleighBlockFading_fast, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape

In [8]:
def get_layer_shapes(start, end, num_steps):
    shapes = [start]
    diff = (end-start)/(num_steps-1)
    # Always start with a full dense layer
    for i in range(1,num_steps):
        shapes.append(int(start + i*diff))
    return shapes

In [9]:
def make_complex_n_layer_lr_tanh_tapering_model(M, R, sigma, \
                                                hl_activation_func, \
                                                ol_activation_func, \
                                                num_layers):
    ### Initialising Parameters
    k = np.log2(M) # Number of bits needed to represent M 
                   # messages
    Nc = int(round(k/R)) # Number of bit being used to represent
                        # channel symbols being used 
                        # Number of complex channel uses
    Nr = Nc*2 # Number of real channel uses

    ### Defining Layers
    ## TRANSMITTER
    tx_shapes = get_layer_shapes(M, Nr, num_layers)
    input_message = Input(shape=(M,), name="input")
    # Hidden Tx layers
    tx1 = Dense(tx_shapes[0],activation=hl_activation_func, \
                name="tx1")(input_message)
    for i in range(1,num_layers-1):
        tx1 = Dense(tx_shapes[i],activation=hl_activation_func, \
                    name=("tx"+str(i+1)))(tx1)
    # Final layer with a different activation function to capture non
    # linearity
    tx_n = Dense(tx_shapes[-1],activation=ol_activation_func, \
                 name=("tx"+str(num_layers)))(tx1)
    # Reshape it to complex channel symbols
    tx_complex = Lambda(lambda x : K.reshape(x, (-1,Nc,2)),
                       output_shape=(Nc,2), \
                        name="tx_reshape")(tx_n)

    # Normalisation Layer
    tx_norm = Lambda(lambda x : K.l2_normalize(x,axis=2),
                     output_shape=(Nc,2), name="tx_norm")\
                        (tx_complex)
    tx_norm_scaled = Lambda(lambda x : K.tf.multiply(np.float32(np.sqrt(Nr)), x),
                      output_shape=(Nr,), name="tx_norm_scaled")\
                        (tx_norm)
    
    # Add Noise 
    noise = GaussianNoise2(sigma)(tx_norm_scaled)

    ## RECIEVER
    # Flatten the input
    noise_flat = Lambda(lambda x : K.reshape(x, (-1,Nr)),
                       output_shape=(Nr,),\
                        name="noise_flat")(noise)
    # First layer with the different activation function
    # to capture non-linearity and for symmetry with the 
    # transmitter.
    rx1 = Dense(tx_shapes[-2],activation=ol_activation_func, name="rx1")\
                (noise_flat)
    # Hidden Rx Layers
    if(num_layers >= 3):
        layer_ind = -3
    else:
        layer_ind = -2
    rx_i = Dense(tx_shapes[layer_ind],activation=hl_activation_func, \
                name="rx2")(rx1)
    for i in range(2,num_layers):
        ind = max(0,num_layers - 2 - i)
        rx_i = Dense(tx_shapes[ind],activation=hl_activation_func, \
                    name=("rx"+str(i+1)))(rx_i)
    # Dense layer with softmax activation
    rx_softmax = Dense(tx_shapes[0],activation='softmax', \
                       name="rx_softmax")(rx_i)
    
    # Select the symbols with the maximum probabilities
    ml_symbs = MostLikelySymbol()(rx_softmax)
    
    ###Defining the models
    autoencoder = Model(input_message, rx_softmax)
    ## Model the Tx and Rx seperately as well
    # Model the Tx
    transmitter = Model(input_message, tx_norm_scaled)
    # Model the Tx plus the noise
    channel_sym_with_noise = Model(input_message, noise)
    channel_symbol = Input(shape=(Nr,))
    # Take the last layer of the autoencoder model
    reciever_layers = autoencoder.layers[-(num_layers+1)](channel_symbol)
    for i in range(num_layers):
        reciever_layers = autoencoder.layers[-(num_layers-i)](reciever_layers)

    # Create a model of the reciever
    reciever = Model(channel_symbol, reciever_layers)
    autoencoder_symbs = Model(input_message,ml_symbs) 
    
    # Compile the model
    autoencoder.compile(loss='categorical_crossentropy',
                        optimizer="adam")
    return autoencoder, transmitter, reciever,\
            autoencoder_symbs, k, Nc, Nr

In [91]:
in_ = np.random.rand(5,1,2)
in_T = K.constant(in_)
sess = tf.Session()
with sess.as_default():
    in_T.eval(session=sess)
in_T.eval(session=sess)

array([[[0.2953959 , 0.9147259 ]],

       [[0.80803007, 0.83845335]],

       [[0.3660615 , 0.46681798]],

       [[0.7871175 , 0.38486257]],

       [[0.73017544, 0.7195798 ]]], dtype=float32)

In [209]:
fade(in_T, 1, 0).shape

TensorShape([Dimension(5), Dimension(1), Dimension(2)])

In [247]:
def custom_mult(in_,col_sel,h):
    tmp = K.tf.multiply(col_sel,h)
    tmp_expand = K.expand_dims(tmp,axis=1)
    return K.tf.multiply(in_,tmp_expand)

def complex_div(in_,h):
    o1 = K.constant(np.array([1,0]))
    o2 = K.constant(np.array([0,1]))
    h_swap = K.tf.reverse(h,[1])
    in_swap = K.tf.reverse(in_,[2])

    t1 = custom_mult(in_,o1,h)          # real*real
    t2 = custom_mult(in_swap,o1,h_swap) # imaginary*imaginary
    t3 = custom_mult(in_swap,o2,h)      # real*imaginary
    t4 = custom_mult(in_,o2,h_swap)     # imaginary*real

    total1 = K.tf.math.add(t1,t2)        # xr*hr-xi*hi
    total2 = K.tf.math.subtract(t4,t3)   # xr*hi-xi*hr
    total = K.tf.math.add(total1,total2) # xr*hr+xi*hi + xr*hi-xi*hr

    scale_factor = K.sum(K.tf.math.square(hT), axis=1) # hr^2 + hi^2
    scale_factor_expanded_twice = K.expand_dims(K.expand_dims(scale_factor, axis=1), axis=2)
    return K.tf.math.divide(total,scale_factor_expanded_twice)  

In [232]:
sigma_m = 0.33
# hT = K.random_normal(shape=(K.shape(in_T)[0],K.shape(in_T)[2]),
#                      mean=0.,
#                      stddev=sigma_m)
hT = K.constant(np.random.rand(5,2))

In [250]:
in_T.eval(session=sess)

array([[[0.2953959 , 0.9147259 ]],

       [[0.80803007, 0.83845335]],

       [[0.3660615 , 0.46681798]],

       [[0.7871175 , 0.38486257]],

       [[0.73017544, 0.7195798 ]]], dtype=float32)

In [251]:
hT.eval(session=sess)

array([[0.29887655, 0.8344373 ],
       [0.73821604, 0.4937612 ],
       [0.3264164 , 0.9304078 ],
       [0.7905989 , 0.9422098 ],
       [0.4626853 , 0.6447809 ]], dtype=float32)

In [249]:
complex_div(in_T,hT).eval(session=sess)

array([[[ 1.0839542 ,  0.03424172]],

       [[ 1.2811154 ,  0.27889976]],

       [[ 0.56965226, -0.19359002]],

       [[ 0.6510521 , -0.28910375]],

       [[ 1.2730824 , -0.2188945 ]]], dtype=float32)

In [235]:
hT.eval(session=sess)

array([[0.29887655, 0.8344373 ],
       [0.73821604, 0.4937612 ],
       [0.3264164 , 0.9304078 ],
       [0.7905989 , 0.9422098 ],
       [0.4626853 , 0.6447809 ]], dtype=float32)

In [222]:
scale_factor = K.sum(K.tf.math.square(hT), axis=1)
scale_factor.shape

TensorShape([Dimension(5)])

AWGN alternating model has a single dense ReLu layer, I'm going to use the topology I found to be best in the other notebook, two layers, tapered 

In [256]:
M = 8
Nc = 4
sigma_m = 0.33
sigma_a = 1

k = np.log2(M) # Number of bits needed to represent M 
               # messages
Nr = Nc*2 # Number of real channel uses

### Defining Layers
## TRANSMITTER
tx_input_message = Input(shape=(M,), name="input")
tx1 = keras.layers.Embedding(M, M*M)(tx_input_message)
tx1 = keras.layers.ELU(alpha=1.0)(tx1)
tx2 = Dense(Nr,activation="linear", name="tx2")(tx1)

# Reshape it to complex channel symbols
tx_complex = Lambda(lambda x : K.reshape(x, (-1,Nc,2)),
                   output_shape=(Nc,2), \
                    name="tx_reshape")(tx2)

# Normalisation Layer
tx_norm = Lambda(lambda x : K.l2_normalize(x,axis=2),
                 output_shape=(Nc,2), name="tx_norm")\
                    (tx_complex)
tx_norm_scaled = Lambda(lambda x : K.tf.multiply(np.float32(np.sqrt(Nr)), x),
                  output_shape=(Nc,2), name="tx_norm_scaled")\
                    (tx_norm)
# Add AWGN Noise 
noise = RayleighBlockFading_fast(sigma_m,sigma_a)(tx_norm_scaled)

## RECIEVER
rx_input_message = Input(shape=(M,), name="input")
# Flatten the input
noise_flat = Lambda(lambda x : K.reshape(x, (-1,Nr)),
                   output_shape=(Nr,),\
                    name="noise_flat")(rx_input_message)
# First layer with the different activation function
# to capture non-linearity and for symmetry with the 
# transmitter.
h1 = Dense(20,activation="tanh", name="h1")\
            (noise_flat)
h_hat = Dense(2,activation="linear", name="h_hat")\
            (h1)
y_over_h = Lambda(lambda x : complex_div(x,h_hat),
                  output_shape=(Nr,),\
                  name="y_over_h")(noise_flat)
# rx1 = Dense(M,activation="relu", name="rx1")\
#             (y_over_h)
# # Dense layer with softmax activation
# rx_softmax = Dense(M,activation='softmax', \
#                    name="rx_softmax")(rx1)
# # Select the symbols with the maximum probabilities
# ml_symbs = MostLikelySymbol()(rx_softmax)

# ###Defining the models
# # Model the Tx
# transmitter = Model(tx_input_message, tx_norm_scaled)
# # Model the Tx plus the noise
# channel_sym_with_noise = Model(tx_input_message, noise)
# # Model the Rx
# receiver = Model(rx_input_message,rx_softmax)
# receiver_symbs = Model(rx_input_message,ml_symbs)

# # Compile the model
# transmitter.compile(loss='categorical_crossentropy',
#                     optimizer="adam")
# receiver.compile(loss='categorical_crossentropy',
#                     optimizer="adam")

ValueError: 'axis'[0] = 2 is out of valid range [0, 1 for 'y_over_h_3/ReverseV2_1' (op: 'ReverseV2') with input shapes: [?,8], [1] and with computed input tensors: input[1] = <2>.

In [187]:
def get_paper_rbf_rx_and_tx_models(M, Nc, sigma):
    k = np.log2(M) # Number of bits needed to represent M 
                   # messages
    Nr = Nc*2 # Number of real channel uses

    ### Defining Layers
    ## TRANSMITTER
    tx_input_message = Input(shape=(M,), name="input")
    tx1 = keras.layers.Embedding(M, M*M)(tx_input_message)
    tx1 = keras.layers.ELU(alpha=1.0)(tx1)
    tx2 = Dense(Nr,activation="linear", name="tx2")(tx1)

    # Reshape it to complex channel symbols
    tx_complex = Lambda(lambda x : K.reshape(x, (-1,Nc,2)),
                       output_shape=(Nc,2), \
                        name="tx_reshape")(tx2)

    # Normalisation Layer
    tx_norm = Lambda(lambda x : K.l2_normalize(x,axis=2),
                     output_shape=(Nc,2), name="tx_norm")\
                        (tx_complex)
    tx_norm_scaled = Lambda(lambda x : K.tf.multiply(np.float32(np.sqrt(Nr)), x),
                      output_shape=(Nc,2), name="tx_norm_scaled")\
                        (tx_norm)
    # Add AWGN Noise 
    noise = GaussianNoise2(sigma)(tx_norm_scaled)

    ## RECIEVER
    rx_input_message = Input(shape=(M,), name="input")
    # Flatten the input
    noise_flat = Lambda(lambda x : K.reshape(x, (-1,Nr)),
                       output_shape=(Nr,),\
                        name="noise_flat")(rx_input_message)
    # First layer with the different activation function
    # to capture non-linearity and for symmetry with the 
    # transmitter.
    rx1 = Dense(M,activation="relu", name="rx1")\
                (noise_flat)
    # Dense layer with softmax activation
    rx_softmax = Dense(M,activation='softmax', \
                       name="rx_softmax")(rx1)
    # Select the symbols with the maximum probabilities
    ml_symbs = MostLikelySymbol()(rx_softmax)

    ###Defining the models
    # Model the Tx
    transmitter = Model(tx_input_message, tx_norm_scaled)
    # Model the Tx plus the noise
    channel_sym_with_noise = Model(tx_input_message, noise)
    # Model the Rx
    receiver = Model(rx_input_message,rx_softmax)
    receiver_symbs = Model(rx_input_message,ml_symbs)

    # Compile the model
    transmitter.compile(loss='categorical_crossentropy',
                        optimizer="adam")
    receiver.compile(loss='categorical_crossentropy',
                        optimizer="adam")    
    return transmitter, channel_sym_with_noise, reciever,\
                receiver_symbs

In [187]:
def get_paper_awgn_rx_and_tx_models(M, Nc, sigma, activation_func):
    k = np.log2(M) # Number of bits needed to represent M 
                   # messages
    Nr = Nc*2 # Number of real channel uses

    ### Defining Layers
    ## TRANSMITTER
    tx_input_message = Input(shape=(M,), name="input")
    tx1 = keras.layers.Embedding(M, M*M)(tx_input_message)
    tx1 = keras.layers.ELU(alpha=1.0)(tx1)
    tx2 = Dense(Nr,activation="linear", name="tx2")(tx1)

    # Reshape it to complex channel symbols
    tx_complex = Lambda(lambda x : K.reshape(x, (-1,Nc,2)),
                       output_shape=(Nc,2), \
                        name="tx_reshape")(tx2)

    # Normalisation Layer
    tx_norm = Lambda(lambda x : K.l2_normalize(x,axis=2),
                     output_shape=(Nc,2), name="tx_norm")\
                        (tx_complex)
    tx_norm_scaled = Lambda(lambda x : K.tf.multiply(np.float32(np.sqrt(Nr)), x),
                      output_shape=(Nc,2), name="tx_norm_scaled")\
                        (tx_norm)
    # Add AWGN Noise 
    noise = GaussianNoise2(sigma)(tx_norm_scaled)

    ## RECIEVER
    rx_input_message = Input(shape=(M,), name="input")
    # Flatten the input
    noise_flat = Lambda(lambda x : K.reshape(x, (-1,Nr)),
                       output_shape=(Nr,),\
                        name="noise_flat")(rx_input_message)
    # First layer with the different activation function
    # to capture non-linearity and for symmetry with the 
    # transmitter.
    rx1 = Dense(M,activation="relu", name="rx1")\
                (noise_flat)
    # Dense layer with softmax activation
    rx_softmax = Dense(M,activation='softmax', \
                       name="rx_softmax")(rx1)
    # Select the symbols with the maximum probabilities
    ml_symbs = MostLikelySymbol()(rx_softmax)

    ###Defining the models
    # Model the Tx
    transmitter = Model(tx_input_message, tx_norm_scaled)
    # Model the Tx plus the noise
    channel_sym_with_noise = Model(tx_input_message, noise)
    # Model the Rx
    receiver = Model(rx_input_message,rx_softmax)
    receiver_symbs = Model(rx_input_message,ml_symbs)

    # Compile the model
    transmitter.compile(loss='categorical_crossentropy',
                        optimizer="adam")
    receiver.compile(loss='categorical_crossentropy',
                        optimizer="adam")    
    return transmitter, channel_sym_with_noise, reciever,\
                receiver_symbs

### Loading Models, Data and Results

In [ ]:
# # Get a set of data for a particular M
# total_size = 10000000
# all_one_hot_messages256 = np.diag(np.ones(256))


# # Automatically saves the data for m to a filepath of
# # './data/data${M}.npy'
# t2 = time()
# print(f"Finished 16 in {t2-t1}s")
# func_data256, file_path256, all_one_hot_messages256 = get_data_set(256, total_size)
# print(f"Finished 256 in {time()-t2}s")

# # Don't use this function unless it's for a new M, just
# # load the data you have calculated other times.
# # This makes results more comparible and saves time.

In [ ]:
# Load the data calculated from previous runs
all_one_hot_messages256 = np.diag(np.ones(256))
data256 = np.load('./data/data256.npy')

In [ ]:
# Splitting into training, testing and validation sets
train_data256, test_data256 = train_test_split(data256, \
                                         train_size=0.8)
train_data256, valid_data256 = train_test_split(train_data256, \
                                         train_size=0.9)
print(f"train_data256.shape = {train_data256.shape}")
print(f"valid_data256.shape = {valid_data256.shape}")
print(f"test_data256.shape = {test_data256.shape}")

### Running Code
##### Getting a supervised model for AWGN

In [ ]:
# (8,8)
M = 2**8 # Number of one hot encoded messages
R = 2 # R = k/n_r
sigma = get_noise_sigma(7, Rc=R)
hl_activation_func = keras.layers.advanced_activations.LeakyReLU()
hl_activation_func.__name__ = 'leakyrelu'
ol_activation_func = "tanh"
num_layers = 2

autoencoder8_8_tap_2l, transmitter8_8_tap_2l, reciever8_8_tap_2l, \
    autoencoder_symbs8_8_tap_2l, k8_8_tap_2l, Nc8_8_tap_2l, Nr8_8_tap_2l \
    = make_complex_n_layer_lr_tanh_tapering_model(M, R, sigma, \
                                                  hl_activation_func, \
                                                  ol_activation_func, \
                                                  num_layers)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

In [ ]:
# # Previous 0.1763
# autoencoder8_8_tap_2l.fit(train_data256, train_data256,
#                        epochs=1000,
#                        batch_size=1000,
#                        shuffle=True,
#                        validation_data=(valid_data256,
#                                         valid_data256),
#                        callbacks=[es])

In [ ]:
autoencoder8_8_tap_2l.load_weights('./models/autoencoder8_8_tap_2l3.3856e-06.h5', by_name=True)

##### Getting a supervised model for RBF

##### Getting an  alternating model for AWGN

##### Getting an  alternating model for RBF